# test the classifier


1. **Initialization**: Load the trained model (`model.p`) using pickle, initialize the MediaPipe Hands module for hand landmark detection, and define a dictionary mapping label indices to characters.

2. **Webcam Initialization**: Start capturing video frames from the webcam using OpenCV.

3. **Frame Processing**: Read a frame from the webcam, convert it to RGB color space, and process it to detect hand landmarks using the MediaPipe Hands module.

4. **Hand Landmark Detection**: If hand landmarks are detected, draw them on the frame. Extract the normalized coordinates of hand landmarks and append them to the `data_aux` list.

5. **Bounding Box and Label**: Determine the bounding box coordinates based on the detected hand landmarks and draw a bounding box around the detected hand gesture on the frame. Make predictions using the trained model based on the extracted hand landmark data. Display the predicted character label near the bounding box.

6. **Display Frame**: Display the processed frame with the bounding box and predicted label.

7. **User Input Handling**: Check for user input to exit the loop (`'q'` key).

8. **Error Handling**: Catch any exceptions that occur during execution and print them.

9. **Cleanup**: Release the webcam and close all OpenCV windows when the program ends.


#### Version 1

In [1]:
import pickle
import cv2
import mediapipe as mp
import numpy as np

In [6]:
model_dict = pickle.load(open('./all_models/Random_Forest_Classifier_model.p', 'rb'))
model = model_dict['model']

In [7]:
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

In [11]:
hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.3)
# labels from 0 to 12
labels_dict = {0:'A', 1:'B', 2:'C', 3:'D', 4:'E', 5:'F', 6:'G', 7:'H', 8:'I', 9:'J', 10:'K', 11:'L', 12:'M', 13:'N', 14:'O', 15:'P', 16:'Q', 17:'R', 18:'S', 19:'T', 20:'U', 21:'V', 22:'W', 23:'X', 24:'Y', 25:'Z'}

- mp_hands.Hands(): This creates an instance of the MediaPipe Hands module, which is used for detecting hand landmarks in images or video frames.

- static_image_mode=True: This parameter indicates whether the module should process images as static images or video frames. When set to True, it indicates that the module will process static images (one-time analysis) rather than continuous video frames.

- min_detection_confidence=0.3: This parameter sets the minimum confidence score required for a hand landmark to be considered detected. Hand landmarks with confidence scores below this threshold will be ignored. In this case, a confidence score of 0.3 is used, meaning that hand landmarks must have a confidence score of at least 30% to be considered valid detections.

In [18]:
import cv2

# Initialize the webcam
cap = cv2.VideoCapture(0)

while True:
    try:
        # Data and variables initialization
        data_aux = []
        x_ = []
        y_ = []

        # Read a frame from the webcam
        ret, frame = cap.read()

        # Get the dimensions of the frame
        H, W, _ = frame.shape

        # Convert the frame to RGB color space
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Process the frame to detect hand landmarks
        results = hands.process(frame_rgb)

        # If hand landmarks are detected
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(
                    frame,  # image to draw
                    hand_landmarks,  # model output
                    mp_hands.HAND_CONNECTIONS,  # hand connections
                    mp_drawing_styles.get_default_hand_landmarks_style(),
                    mp_drawing_styles.get_default_hand_connections_style())

                for i in range(len(hand_landmarks.landmark)):
                    x = hand_landmarks.landmark[i].x
                    y = hand_landmarks.landmark[i].y

                    x_.append(x)
                    y_.append(y)

                for i in range(len(hand_landmarks.landmark)):
                    x = hand_landmarks.landmark[i].x
                    y = hand_landmarks.landmark[i].y
                    data_aux.append(x - min(x_))
                    data_aux.append(y - min(y_))

            x1 = int(min(x_) * W) - 10
            y1 = int(min(y_) * H) - 10

            x2 = int(max(x_) * W) - 10
            y2 = int(max(y_) * H) - 10

            # Make predictions using the model
            prediction = model.predict([np.asarray(data_aux)])
            prediction_prob = model.predict_proba([np.asarray(data_aux)])

            # If predictions are available
            if prediction.shape[0] > 0:
                predicted_label_index = int(prediction[0])
                predicted_character = labels_dict.get(predicted_label_index, "Unknown")
                prediction_probability = np.max(prediction_prob)  # Get the maximum probability
            
            else:
                predicted_character = "Unknown"
                prediction_probability = 0.0  # Set probability to zero for unknown prediction


            # Draw bounding box and label on the frame
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 0), 4)
            if prediction_probability > 0.2:
                cv2.putText(frame, f'{predicted_character} ({prediction_probability:.2f})', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 0, 0), 3, cv2.LINE_AA)

        # Display the frame
        cv2.imshow('frame', frame)

        # Check for user input to exit the loop
        c = cv2.waitKey(1)
        if c == ord('q'):
            break

    except Exception as e:
        # Print the exception
        print("An error occurred:", str(e))

# Release the webcam and close all windows
cap.release()
cv2.destroyAllWindows()


In [21]:
        cv2.imshow('frame', frame)
